1RSB
===

\begin{align*}
p\left(f\right)  &= \sum_{f_{1},\dots,f_{k}}\delta\left(f;\min_{j=1,\dots,k}\left|f_{j}\right|\prod_{j=1}^{k}\text{sign}\left(f_{j}\right)\right)\prod_{j=1}^{k}q_j\left(f_{j}\right)e^{-y F_{ai}}\\
q(f) &= \sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b) \delta\left(f-s-\sum_{b=1,\dots,d}f_b\right)e^{-y F_{ia}}\\
\end{align*}

where
\begin{align*}
F_{ai} &= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\prod_{j=1}^kf_{j}\right)\\
 F_{ia} &=\left|s + \sum_{b=1}^d f_b\right| - \sum_{b=1}^d|f_b|
\end{align*}



In [1]:
using OffsetArrays

const ∏ = prod
const ∑ = sum

sum (generic function with 14 methods)

In [2]:
function iter_slow_factor(Q, n, y=0.0)
    k = length(Q)
    p = OffsetVector(zeros(2N+1), -N:N)
    for fs in Iterators.product(fill(-N:N,k)...)
        Fai = -2*minimum(abs.(fs))*(∏(fs) < 0)
        w = ∏(q[f1] for (q,f1) ∈ zip(Q,fs)) * exp(-y*Fai)
        f = minimum(abs.(fs))*sign(∏(fs))
        p[clamp(f, -N, N)] += w
    end
    p ./ sum(p)
end

function iter_slow_var(P, s, N, y=0.0) 
    q = OffsetVector(zeros(2N+1), -N:N)
    for fs in Iterators.product(fill(-N:N, length(P))...)
        f = sum(fs) + s
        Fia = abs(f) - sum(abs.(fs)) 
        w = ∏(p[f1] for (p,f1) ∈ zip(P,fs)) * exp(-y*Fia)
        q[clamp(f, -N, N)] += w 
    end
    q ./ sum(q)
end

iter_slow_var (generic function with 2 methods)

Simplifications
--

$F_{ai}$ can be simplified: 

\begin{align}
F_{ai} &= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\prod_{j=1}^kf_{j}\right)\\
&= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(\textrm{sign}\left(-\prod_{j=1}^kf_{j}\right)\right)\\
&= -2\min_{j=1,\dots,k}|f_{j}|\Theta\left(-\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(\prod_{j=1}^kf_{j}\right)\right)\\
&= -2\left|\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(-\prod_{j=1}^kf_{j}\right)\right|\Theta\left(-\min_{j=1,\dots,k}|f_{j}|\textrm{sign}\left(\prod_{j=1}^kf_{j}\right)\right)\\
&= 2f\Theta(-f)
\end{align}

So

\begin{align*}
p\left(f\right)= & \sum_{f_{1},\dots,f_{k}}\delta\left(f;\min_{j=1,\dots,k}\left|f_{j}\right|\prod_{j=1}^{k}\text{sign}\left(f_{j}\right)\right)\prod_{j=1}^{k}q_j\left(f_{j}\right)e^{-y 2 f \Theta(-f)}\\
\end{align*}

To compute $p$, define

\begin{align*}
a_k(f) &= p\left(\textrm{sign}\left(\prod_{i=1}^k f_i\right)=\textrm{sign}(f) \wedge |f_1|,\dots,|f_k| \ge |f|\right)\\
\end{align*}

$a_k$ satisfies the recursion

\begin{align*}
a_0(f) &= \delta(\textrm{sign}(f)-1)\\
a_k(f) &= a_{k-1}(f) \sum_{f_k\geq |f|}q_k(f_k) + a_{k-1}(-f) \sum_{f_k \leq -|f|} q_k(f_k)\\
\end{align*}

Then we finally have

$$p_k(f) = \cases{(a_k(f)-a_k(f+\textrm{sign}(f)))e^{-y2f\Theta(-f)} & for $f\neq0$\\
           1-\prod_{j=1}^k(1-q_j(0)) & for $f=0$}$$


In [3]:
function iter_factor(Q, N, y=0)
    a = OffsetVector(zeros(2N+3), -N-1:N+1)
    a[1:N] .= 1
    for q ∈ Q
        sp, sm = 0.0, 0.0
        for f=N:-1:1
            ap, am = a[f], a[-f]
            sp += q[f]; sm += q[-f]
            #sp, sm = ∑(q[f:N]), ∑(q[-N:-f])
            a[+f] = ap*sp + am*sm
            a[-f] = am*sp + ap*sm
        end
    end
    
    p = OffsetVector(zeros(2N+1), -N:N)
    for f = 1:N
        p[+f] = (a[+f]-a[+f+1])
        p[-f] = (a[-f]-a[-f-1])*exp(2y*f)
    end
    p[0] = 1-∏(1-q[0] for q ∈ Q)
    p ./ sum(p)
end


iter_factor (generic function with 2 methods)

Comparison
--

In [4]:
N=10
y=0.1
Q=[(p=OffsetArray(zeros(2N+1),-N:N); p[-9:9] .= rand(19); p ./=sum(p); p) for i=1:3]
[iter_factor(Q,N,1) iter_slow_factor(Q,N,1)]

21×2 Array{Float64,2}:
 0.0          0.0
 0.549897     0.549897
 0.325407     0.325407
 0.0955109    0.0955109
 0.0228723    0.0228723
 0.0051898    0.0051898
 0.000983216  0.000983216
 9.80494e-5   9.80494e-5
 3.31257e-5   3.31257e-5
 6.10358e-6   6.10358e-6
 5.48222e-7   5.48222e-7
 8.2551e-7    8.2551e-7
 6.06589e-7   6.06589e-7
 2.433e-7     2.433e-7
 3.29828e-7   3.29828e-7
 2.35242e-7   2.35242e-7
 1.41497e-7   1.41497e-7
 7.88596e-8   7.88596e-8
 3.65998e-8   3.65998e-8
 8.38734e-9   8.38734e-9
 0.0          0.0

The expression for $F_{ia}$ is

\begin{align*}
    F_{ia} &=\left|s + \sum_{b=1}^d f_b\right| - \sum_{b=1}^d|f_b|
\end{align*}
and

\begin{align*}
q(f) &= \sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b) \delta\left(f-s-\sum_{b=1}^df_b\right)e^{-y F_{ia}}\\
&= e^{-y |f|}\sum_{f_1,\dots,f_d} \prod_{b=1,\dots,d} p_b(f_b)e^{y|f_b|} \delta\left(f-s-\sum_{b=1}^df_b\right)\\
\end{align*}


To compute $q_d$, note that

\begin{align}
q(f) &= q_d(f)e^{-y|f|}
\end{align}

where $q_d$ satisfies
\begin{align}
q_0(f) & = \delta(f-s)\\
q_d(f) & = \sum_{f_d} q_{d-1}(f-f_d) p_d(f_d)e^{-y|f_d|} 
\end{align}

In [5]:
function ⊛(p1, p2)
    N = max(lastindex(p1), lastindex(p2))
    q = OffsetArray(zeros(2N+1),-N:N)
    for f1 in eachindex(p1)
        for f2 in eachindex(p2)
            q[clamp(f1+f2, -N, N)] += p1[f1]*p2[f2]
        end
    end
    q
end

function iter_var(P, s, N, y=0)
    q = OffsetVector(zeros(2N+1), -N:N)
    q[s]=1
    w = exp.(y .* abs.(eachindex(q)))
    for p ∈ P
        q = q ⊛ (p .* w)
    end
    q ./= w
    q ./ sum(q)
end

iter_var (generic function with 2 methods)

Comparison
--

In [6]:
N=13
y=1.0
s=1
P=[(p=OffsetArray(zeros(2N+1),-N:N); p[-5:5] .= rand(11); p ./=sum(p); p) for i=1:3]
[iter_var(P,s,N,1) iter_slow_var(P,s,N,1)]

27×2 Array{Float64,2}:
 4.86083e-6   3.52054e-6
 3.78434e-6   3.78436e-6
 6.96855e-6   6.96858e-6
 1.05435e-5   1.05436e-5
 1.33342e-5   1.33342e-5
 4.09591e-5   4.09593e-5
 0.000258328  0.000258329
 0.00140985   0.00140986
 0.00818762   0.00818765
 0.058468     0.0584682
 0.121281     0.121281
 0.133456     0.133457
 0.216233     0.216234
 ⋮            
 0.047193     0.0471931
 0.0358089    0.0358091
 0.0214961    0.0214962
 0.0124152    0.0124153
 0.00630847   0.00630849
 0.000963007  0.000963011
 9.8602e-5    9.86024e-5
 2.03157e-5   2.03158e-5
 3.54103e-6   3.54105e-6
 1.93754e-6   1.93754e-6
 1.40178e-6   1.40179e-6
 4.1637e-6    1.48962e-6

In [7]:
using Distributions, ProgressMeter

uni(N) = (p=OffsetVector(ones(2N+1), -N:N); p ./= sum(p))
residual(x) = (p=OffsetVector((x .* eachindex(x))[1:end], 0:lastindex(x)-1); p./=sum(p))

function RSB(Λ, K; maxiter=100, popsize=1000, N=10, popP = [uni(N) for i=1:popsize], popQ = [uni(N) for i=1:popsize], y=0)
    Λ1 = residual(Λ)
    K1 = residual(K)

    distΛ1 = DiscreteNonParametric(collect(eachindex(Λ1)), Λ1.parent)
    distK1 = DiscreteNonParametric(collect(eachindex(K1)), K1.parent)

    @showprogress for t = 1:maxiter
        for i = eachindex(popQ)
            d = rand(distΛ1)
            k = rand(distK1)
            Q = rand(popQ, k)
            P = rand(popP, d)
            s = rand((-1,1))
            popQ[i] = iter_var(P, s, N, y)
            popP[i] = iter_factor(Q, N, y)
        end
    end
    popP, popQ
end

RSB (generic function with 1 method)

In [8]:
Λ = OffsetVector([0,0,1], 0:2)
K = OffsetVector([0,0,0,1], 0:3)


N=6
popsize=10000
popQ = [OffsetVector(ones(2N+1)/(2N+1),-N:N) for i=1:popsize];
popP = [OffsetVector(ones(2N+1)/(2N+1),-N:N) for i=1:popsize];

In [10]:
popP, popQ = RSB(Λ,K; N=6, maxiter=200, popsize=popsize, popQ=popQ, popP=popP, y=1.0);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


In [11]:
using StatsBase

sort(proportionmap(argmax.(popP)))

OrderedCollections.OrderedDict{Int64,Float64} with 7 entries:
  -3 => 0.0005
  -2 => 0.0395
  -1 => 0.2498
  0  => 0.423
  1  => 0.2475
  2  => 0.0395
  3  => 0.0002

In [12]:
popP

10000-element Array{OffsetArray{Float64,1,Array{Float64,1}},1}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 ⋮
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [13]:
using SparseArrays
q=OffsetVector(zeros(2N+1),-N:N); q[1]=1

Q=[q for i=1:2]; 

p=iter_slow_factor(Q, N);
@show p'
q1=iter_slow_var([p],N);
@show q1'


p' = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0]


LoadError: MethodError: no method matching iter_slow_var(::Array{OffsetArray{Float64,1,Array{Float64,1}},1}, ::Int64)
Closest candidates are:
  iter_slow_var(::Any, ::Any, !Matched::Any) at In[2]:13
  iter_slow_var(::Any, ::Any, !Matched::Any, !Matched::Any) at In[2]:13

1RSB equations for Max-Sum (Survey Propagation) at finite parameter $y$
---
We consider the auxiliary statistical physics model
\begin{align}
    \mathbb{P}_y(\{u_{ai},v_{ia}\})=\frac{1}{\Theta(y)}\mathbb{I}\left(\{u_{ai},v_{ia}\}\text{sat Max-Sum}\right)e^{-yF(\{u_{ai},v_{ia}\})}
\end{align}
in which the variables are the Max-Sum messages living on the edges, and the constraints (that can be put in a factorized form) enforces the Max-Sum equations to be satisfied. Each Max-Sum solution represents a cluster. The Max-Sum solutions are weighted according to their Bethe free energy, which corresponds to the minimal energy (inside the cluster). In the $y\to\infty$ limit one keeps only the clusters with minimal energy

* The distributions $Q_{ia}(v_{ia})$, $P_{ai}(u_{ai})$ over Max-Sum messages obey the SP(y) equations :
\begin{align}
    Q_{ia}(v_{ia})&=\frac{1}{Z_{ia}}\sum_{\{u_{bi}\}_{b\in\partial i\setminus a}}\delta(v_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}; s_i))e^{-yC_{ia}}\prod_{b\in\partial i\setminus a}P_{bi}(u_{bi}) \\
    P_{ai}(u_{ai})&=\frac{1}{Z_{ai}}\sum_{\{v_{ja}\}_{j\in\partial a\setminus i}}\delta(u_{ai}-f^{MS}_{ai}(\{v_{ja}\}_{j\in\partial a\setminus i}))e^{-yC_{ai}}\prod_{j\in\partial a\setminus i}Q_{ja}(v_{ja})    
\end{align}
where $v_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}; s_i)$, and $u_{ai}-f^{MS}_{ai}(\{v_{ja}\}_{j\in\partial a\setminus i})$ are shorthand notation for the Max-Sum equations
\begin{align}
    v_{ia}(\sigma_i)&=s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i) - C_{ia}\\
    u_{ai}(\sigma_i)&=\max_{\{\sigma_j\}_{j\in\partial a\setminus i}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}v_{ja}(\sigma_j)\right) - C_{ai}
\end{align}
and $C_{ia}$, $C_{ai}$ are the constants in the Max-Sum equations
\begin{align}
    C_{ia}&=\max_{\sigma_i}\left(s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i)\right)\\
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}v_{ja}(\sigma_j)\right)
\end{align}
the constants $Z_{ia}$, $Z_{ai}$ are normalizations of the distributions $Q_{ia}$, $Q_{ai}$ over the Max-Sum messages

* With the parametrization $u_{ai}(\sigma_i) = f_{ai}\sigma_i+f_{0,ai}$ and $v_{ia}(\sigma_i) = g_{ia}\sigma_i + g_{0,ia}$ we get :
\begin{equation}
    f_{0,ai} = -\frac{1}{\beta}\log(2{\rm cosh}(\beta f_{ai})\to-|f_{ai}|
\end{equation}
(and the same for $g_{0,ia}$). Therefore :
\begin{align*}
    C_{ia}&=\max_{\sigma_i}\left(\sigma_i\left(s_i+ \sum_{b\in\partial i\setminus a}f_{bi}\right)\right) + \sum_{b\in\partial i\setminus a}f_{0,bi}\\
    &=\left|s_i+ \sum_{b\in\partial i\setminus a}f_{bi}\right| - \sum_{b\in\partial i\setminus a}|f_{bi}|
\end{align*}
and
\begin{align*}
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}g_{ja}\sigma_j\right)-\sum_{j\in\partial a\setminus i}|g_{ja}|\\
    &= -2\min_{j\in\partial a\setminus i}|g_{ja}|\Theta\left(-\prod_{j\in\partial a\setminus j}g_{ja}\right)
\end{align*}

* We can then write the same 1RSB equations for the distributions $Q_{ia}(g_{ia})$, $P_{ai}(f_{ai})$ over the integer parameters $g_{ia}$, $f_{ai}$:
\begin{align}
    P(f)&=\frac{1}{Z_P(y)}\sum_{g_1,\dots g_k}\delta\left(f-\min_{j=1\dots,k}|g_j|\prod_j{\rm sgn}(g_j)\right)e^{-yC_{ai}(g_1,\dots,g_k)}\prod_j Q_j(g_j) \\
    Q(g)&=\frac{1}{Z_Q(y)}\sum_{f_1,\dots f_d}\delta(g-s-\sum_{b=1}^df_b)e^{-yC_{ia}(f_1,\dots,f_d)}\prod_{b=1}^dP_b(f_b)
\end{align}

* Random graph ensemble
When the source and the factor graph are random variables the 1RSB messages $P_{ai}$, $(a,i)\in E$ become random variables. Let $(a,i)$ be a uniformly chosen edge in a factor graph drawn from the random ensembl, and let $\mathcal{P}$ be the distribution of the message $P_{ai}$ solution of the 1RSB equation written above. The distribution $\mathcal{P}(p)$ obey consistency equation similar to the RS cavity equations:
\begin{align}
    \mathcal{P}^{1RSB}(P)&=\sum_{k}\tilde{P}_k \int\prod_{j=1}^k{\rm d}\mathcal{Q}^{1RSB}(Q_j)\delta(P-F^{SP(y)}(Q_1,\dots,Q_k)) \\
    \mathcal{Q}^{1RSB}(Q) &= \sum_{s}\frac{1}{2}\sum_{d}\tilde{\Lambda_d}\int\prod_{b=1}^d{\rm d}\mathcal{P}^{1RSB}(P_b)\delta(Q-G^{SP(y)}(P_1,\dots,P_d;s))
\end{align}
This equation always admits a trivial fixed point $\mathcal{P}(P)=\sum_f p^{RS}(f)\delta[P-\delta(\cdot-f)]$, $\mathcal{Q}(g)=\sum_g q^{RS}(g)\delta[Q-\delta(\cdot-g)]$, where $p^{RS}, q^{RS}$  is the solution of the RS cavity equation.
In the RS phase, this trivial fixed-point is the unique solution, while in the 1RSB phase, the trivial solution becomes unstable and the above equation admits a non-trivial solution.

Average minimal energy : Free energy of the auxiliary model
---
We want to estimate the Free energy $\mathcal{F}(y)=-\frac{1}{yn}\log\Theta(y)$ of the auxiliary problem defined by the partition function $\Theta(y)$ on a single instance. Then we can average over the random ensemble of instances. In the large $y\to\infty$ limit, we obtain the averaged minimal energy.

On a single instance, once a fixed point of the SP(y) equations has been found on a single instance, one can compute the Bethe free energy of the auxiliary problem:
\begin{align}
    \mathcal{F}(y) = \frac{1}{n}\sum_{a}\mathcal{F}_a(\{Q_{ia}\}_{i\in\partial a};y) + \frac{1}{n}\sum_i\mathcal{F}_i(\{P_{ai}\}_{a\in\partial i};y) - \frac{1}{n}\sum_{(ia)}\mathcal{F}_{(ia)}(P_{ai},Q_{ia};y)
\end{align}
with:
\begin{align}
    \mathcal{F}_a(\{Q_{ia}\}_{i\in\partial a};y) &= -\frac{1}{y}\log\left(\sum_{\{g_{ia}\}_{i\in\partial a}}e^{-yF_a(\{g_{ia}\}_{i\in\partial a})}\prod_{i\in\partial a}Q_{ia}(g_{ia})\right)\\
    \mathcal{F}_i(\{P_{ai}\}_{a\in\partial i};y) &= -\frac{1}{y}\log\left(\sum_{\{f_{ai}\}_{a\in\partial i}}e^{-yF_i(\{f_{ai}\}_{a\in\partial i})}\prod_{a\in\partial i}P_{ai}(f_{ai})\right)\\
    \mathcal{F}_{(ia)}(P_{ai},Q_{ia};y) &= -\frac{1}{y}\log\left(\sum_{f_{ia}, g_{ia}}e^{-yF_{ia}(f_{ia}, g_{ia})}P_{ai}(f_{ai})Q_{ia}(g_{ia})\right)
\end{align}
where $F_a,F_i, F_{ia}$ are the factor, variable and edge terms of the Bethe free energy associated to one fixed point of the Max-Sum equation: 
\begin{align}
    F_a(\{g_{ia}\}_{i\in\partial a}) &= 2\min_{i\in\partial a}|g_{ia}|\Theta(-\prod_i g_{ia})\\
    F_i(\{f_{ai}\}_{a\in\partial i}) &= -|s_i + \sum_{a\in\partial i}f_{ai}| + \sum_a |f_{ai}|\\
    F_{(ia)}(f_{ai},g_{ia}) &= -|f_{ai} + g_{ia}| + |f_{ai}| + |g_{ia}| 
\end{align}
Averaging over the random ensemble of instances:
\begin{align}
    \mathcal{F}^{1RSB}(y) &= \alpha\sum_{k}P_k\int\prod_{i=1}^k{\rm d}\mathcal{Q}^{1RSB}(Q_i)\mathcal{F}_a(Q_1,\dots,Q_k;y) + \sum_d\Lambda_d\sum_s\frac{1}{2}\int\prod_{i=1}^d\mathcal{P}^{1RSB}(P_i)\mathcal{F}_i(P_1,\dots,P_d;s,y) \\
    &- \Lambda'(1)\int\mathcal{F}_{(ia)}\mathcal{P}^{1RSB}(P)\mathcal{Q}^{1RSB}(Q)(P,Q;y)
\end{align}


In [14]:
function overlap_slow_factor(Q, N, y=0.0)
    w = 0.0
    k = length(Q)
    for fs in Iterators.product(fill(-N:N,k)...)
        Fa = 2*minimum(abs.(fs))*(∏(fs) < 0)
        w += ∏(q[f1] for (q,f1) ∈ zip(Q,fs)) * exp(-y*Fa)
    end
    log(w)
end

function overlap_slow_var(P, s, N, y=0.0)
    w = 0.0
    d = length(P)
    for fs in Iterators.product(fill(-N:N,d)...)
        f = sum(fs) + s
        Fi = - abs(f) + sum(abs.(fs)) 
        w += ∏(p[f1] for (p,f1) ∈ zip(P,fs)) * exp(-y*Fi)
    end
    log(w)
end

function overlap_slow_edge(p, q, N, y=0.0)
    w = 0.0
    for f in -N:N
        for g in -N:N
            Fia = abs(f)+abs(g)-abs(f+g)
            w += p[f]*q[g]*exp(-y*Fia)
        end
    end
    log(w)
end

overlap_slow_edge (generic function with 2 methods)

In [20]:
function overlap1RSB(Λ, K; maxiter=100, popsize=1000, N=10, popP = [uni(N) for i=1:popsize], popQ = [uni(N) for i=1:popsize], y=0.0)
    distΛ = DiscreteNonParametric(collect(eachindex(Λ)), Λ.parent)
    distK = DiscreteNonParametric(collect(eachindex(K)), K.parent)
    
    O_factor = 0.0
    O_var = 0.0
    O_edge = 0.0
    @showprogress for t = 1:maxiter
        for i = eachindex(popQ)
            k = rand(distK)
            Q = rand(popQ, k)
            O_factor += overlap_slow_factor(Q, N, y)

            d = rand(distΛ)
            P = rand(popP, d)
            s = rand((-1,1))
            O_var += overlap_slow_var(P, s, N, y)
            
            p = rand(popP)
            q = rand(popQ)
            popQ[i] = iter_var(P, s, N, y)
            popP[i] = iter_factor(Q, N, y)
            O_edge += overlap_slow_edge(p, q, N, y)
        end
    end
    mK = sum(k*K[k] for k=eachindex(K))
    mΛ = sum(d*Λ[d] for d=eachindex(Λ))
    α = mΛ/mK
    - α*O_factor - O_var + mΛ*O_edge
end

overlap1RSB (generic function with 1 method)

In [21]:
Λ = OffsetVector([0,0,1], 0:2)
K = OffsetVector([0,0,0,1], 0:3)
overlap1RSB(Λ,K; N=6, maxiter=50, popsize=50)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


4.3820502781956605e-13